# Forced Alignment

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensein/senselab/blob/main/tutorials/audio/forced_alignment.ipynb)

Forced alignment matches parts of speech in an orthographic transcription to timestamps in a corresponding audio file. This enables annotation of transcriptions for large-scale audio datasets. 

Forced alignment can be done in senselab as follows:

In [ ]:
%pip install 'senselab[audio]'

In [ ]:
import os
import platform

from senselab.audio.data_structures import Audio
from senselab.audio.tasks.plotting import play_audio
from senselab.audio.tasks.preprocessing import resample_audios

# Download an audio file for testing
os.makedirs("tutorial_audio_files", exist_ok=True)
url = "https://github.com/sensein/senselab/raw/main/src/tests/data_for_testing/audio_48khz_mono_16bits.wav"
filepath = "tutorial_audio_files/audio_48khz_mono_16bits.wav"

if platform.system() == "Linux":
    os.system(f"wget -O {filepath} {url}")
elif platform.system() == "Darwin":
    os.system(f"curl -L -o {filepath} {url}")
else:
    raise RuntimeError("Unsupported OS")

# Load the audio file from the specified file path as an Audio object.
audio = Audio.from_filepath("tutorial_audio_files/audio_48khz_mono_16bits.wav")

# Resample the audio to 16 kHz
audio = resample_audios([audio], 16000)[0]

# Display audio playback button
play_audio(audio)


In [ ]:

from senselab.utils.data_structures.model import HFModel
from senselab.audio.tasks.speech_to_text.api import transcribe_audios
from senselab.audio.tasks.forced_alignment.forced_alignment import align_transcriptions

# Load a transcription model
whisper_tiny = HFModel(path_or_uri="openai/whisper-tiny")

# Transcribe the audio file
transcription = transcribe_audios(audios=[audio], model=whisper_tiny)[0]

# Align the transcription to the audio waveform
aligned_transcription = align_transcriptions([(audio, transcription)])

# Print the aligned sentences
for aligned_sentence in aligned_transcription[0]:
    print(aligned_sentence)

Senselab's implementation of forced alignment can return alignments at four levels of granularity: utterance, sentence, word, or character. Sentence alignments are always included. Here are examples that include each of these:

In [ ]:
# Utterance and sentence:
levels_to_keep = {"utterance": True, "word": False, "char": False}
aligned_transcription = align_transcriptions([(audio, transcription)], levels_to_keep=levels_to_keep)
print(aligned_transcription)